In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Window_Lab").getOrCreate()

data = [
    ("iPhone", "Mobile", 1000),
    ("Samsung", "Mobile", 800),
    ("Xiaomi", "Mobile", 800), # Цена совпадает с Samsung!
    ("Nokia", "Mobile", 300),
    ("MacBook", "Laptop", 2000),
    ("Dell", "Laptop", 1500),
    ("HP", "Laptop", 1200)
]

columns = ["product", "category", "price"]

df = spark.createDataFrame(data, columns)

df.show()

+-------+--------+-----+
|product|category|price|
+-------+--------+-----+
| iPhone|  Mobile| 1000|
|Samsung|  Mobile|  800|
| Xiaomi|  Mobile|  800|
|  Nokia|  Mobile|  300|
|MacBook|  Laptop| 2000|
|   Dell|  Laptop| 1500|
|     HP|  Laptop| 1200|
+-------+--------+-----+



### Шаг 2: Ранжирование (Ranking) 🏆  
Задача: Найти самые дорогие товары в каждой категории.

In [2]:
# 1. Создаем спецификацию окна
# Разбиваем по категории, сортируем по цене УБЫВАНИЮ (desc)
window_spec_rank = Window.partitionBy("category").orderBy(F.col("price").desc())

# 2. Применяем функции
df_ranked = df \
        .withColumn("row_number", F.row_number().over(window_spec_rank)) \
        .withColumn("rank", F.rank().over(window_spec_rank)) \
        .withColumn("dense_rank", F.dense_rank().over(window_spec_rank))

df_ranked.show()



+-------+--------+-----+----------+----+----------+
|product|category|price|row_number|rank|dense_rank|
+-------+--------+-----+----------+----+----------+
|MacBook|  Laptop| 2000|         1|   1|         1|
|   Dell|  Laptop| 1500|         2|   2|         2|
|     HP|  Laptop| 1200|         3|   3|         3|
| iPhone|  Mobile| 1000|         1|   1|         1|
|Samsung|  Mobile|  800|         2|   2|         2|
| Xiaomi|  Mobile|  800|         3|   2|         2|
|  Nokia|  Mobile|  300|         4|   4|         3|
+-------+--------+-----+----------+----+----------+



### Шаг 3: Сравнение с прошлым (Lag) 📉
Задача: Посчитать разницу в цене между текущим товаром и следующим по дешевизне.

In [4]:
# Сортируем по цене (от дешевых к дорогим)
windows_spec_diff = Window.partitionBy(F.col("category")).orderBy(F.col("price"))

df_diff = df \
            .withColumn("prev_price", F.lag("price", 1).over(window_spec_rank)) \
            .withColumn("price_diff", F.col("price") - F.col("prev_price"))

df_diff.show()
# Первая строка в группе всегда будет иметь NULL в prev_price и diff.

+-------+--------+-----+----------+----------+
|product|category|price|prev_price|price_diff|
+-------+--------+-----+----------+----------+
|MacBook|  Laptop| 2000|      NULL|      NULL|
|   Dell|  Laptop| 1500|      2000|      -500|
|     HP|  Laptop| 1200|      1500|      -300|
| iPhone|  Mobile| 1000|      NULL|      NULL|
|Samsung|  Mobile|  800|      1000|      -200|
| Xiaomi|  Mobile|  800|       800|         0|
|  Nokia|  Mobile|  300|       800|      -500|
+-------+--------+-----+----------+----------+



### Шаг 4: Накопительный итог (Running Total) 📈
Задача: Посчитать сумму цен накопительным итогом (например, для контроля бюджета).

In [5]:
# rowsBetween указывает границы рамки. 
# Window.unboundedPreceding - от самого начала раздела
# Window.currentRow - до текущей строки

window_cum = Window.partitionBy(F.col("category")).orderBy(F.col("price")) \
                .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_cum = df.withColumn("running_total", F.sum(F.col("price")).over(window_cum))

df_cum.show()



+-------+--------+-----+-------------+
|product|category|price|running_total|
+-------+--------+-----+-------------+
|     HP|  Laptop| 1200|         1200|
|   Dell|  Laptop| 1500|         2700|
|MacBook|  Laptop| 2000|         4700|
|  Nokia|  Mobile|  300|          300|
|Samsung|  Mobile|  800|         1100|
| Xiaomi|  Mobile|  800|         1900|
| iPhone|  Mobile| 1000|         2900|
+-------+--------+-----+-------------+



In [6]:
spark.stop()